<h3> Data Cleaning </h3>

In [1]:
import numpy as np
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [20]:
delegates = pd.read_csv("Data/Delegates/cleaned/constitutional_convention_delegates_cleaned.csv").drop('Unnamed: 0', axis = 1)
state_delegates = pd.read_csv("Data/Delegates/cleaned/State_Delegates_cleaned.csv").drop('Unnamed: 0', axis = 1)

In [21]:
final_matching = pd.read_csv('Data/final_matching.csv').drop(['Unnamed: 0', 'Scores'], axis = 1)

In [22]:
spec_names = final_matching[final_matching['Delegates'] != final_matching['Loan Matches']]

# Delegate Asset Database

In [23]:
delegates_assets = delegates[['full name 1', 'full name 2', 'sign']]
state_delegate_assets = state_delegates[['full name 1', 'State', 'Vote']]
state_delegate_assets.rename(columns = {"Vote":"sign", "State":"delegate_state"}, inplace = True)
total_assets = pd.concat([delegates_assets, state_delegate_assets]).reset_index(drop = True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [24]:
#amending existing cases

In [25]:
total_assets['full name 2'] = total_assets['full name 1']
total_assets_full = total_assets

In [26]:
merge_1 = pd.merge(total_assets_full.drop('full name 2', axis = 1), 
                   spec_names, left_on = 'full name 1', 
                   right_on = 'Delegates').drop('full name 1', axis = 1)
merge_2 = pd.merge(total_assets_full.drop('full name 1', axis = 1), 
                   spec_names, left_on = 'full name 2', 
                   right_on = 'Delegates').drop('full name 2', axis = 1)
merged = pd.concat([merge_1, merge_2]).drop_duplicates()

In [27]:
total_assets_full.columns = ['delegate_state', 'delegate name', 'match name', 'sign']
merged.columns =  ['delegate_state', 'sign',  'delegate name', 'match name']

In [28]:
total_assets_final = pd.concat([total_assets_full, merged])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


### Full Name - First/Last Name Mapping

In [29]:
state_firstlast = state_delegates[['full name 1', 'First Name', 'Last Name']].rename(columns = {'full name 1': 'Full Name'})
const_firstlast = delegates[['full name 1', 'first name', 'last name']].rename(columns = {'first name':'First Name',
                                                                                          'last name':'Last Name',
                                                                                          'full name 1': 'Full Name'})
cum_firstlast = pd.concat([state_firstlast, const_firstlast])

# Loans Dataset Matching

In [30]:
#prepare loan dataset
loans = pd.read_csv("Data/Pre1790/cleaned/loan_office_certificates_9_states_cleaned.csv").drop('Unnamed: 0', axis = 1)

loans = loans.rename(columns = {"First Name 1 ":"First Name 1",
                                "Last Name 1 ":"Last Name 1",
                                "Specie Value ":"Specie Value"})

loans['full name 1'] = loans['First Name 1'] + " " + loans['Last Name 1']
loans['full name 2'] = loans['First Name 2'] + " " + loans['Last Name 2']
loans['full name 3'] = loans['First Name 3'] + " " + loans['Last Name 3']

In [31]:
#specie, face and num

In [32]:
loans['index'] = loans.index

loan_small_f1 = loans[['State', 'Year', 'Month', 'Day', 'full name 1', 'Face Value','Specie Value', 'notes', 'original text', 'index']]
loan_small_f2 = loans[['State', 'Year', 'Month', 'Day', 'full name 2', 'Face Value','Specie Value', 'notes', 'original text', 'index']]
loan_small_f3 = loans[['State', 'Year', 'Month', 'Day', 'full name 3', 'Face Value','Specie Value', 'notes', 'original text', 'index']]
loan_office_cert_assets = pd.merge(total_assets_final,
                                   loan_small_f1[loan_small_f1['full name 1'].apply(lambda x: not pd.isnull(x))],
                                   left_on = 'match name', right_on = 'full name 1').drop('full name 1', axis = 1)
add_to_assets = pd.merge(total_assets_final,
                         loan_small_f2[loan_small_f2['full name 2'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 2').drop('full name 2', axis = 1)
loan_office_cert_assets = pd.concat([loan_office_cert_assets, add_to_assets])
add_to_assets = pd.merge(total_assets_final,
                         loan_small_f3[loan_small_f3['full name 3'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 3').drop('full name 3', axis = 1)
loan_office_cert_assets = pd.concat([loan_office_cert_assets, add_to_assets])
loan_office_cert_assets = loan_office_cert_assets.drop_duplicates(['delegate name', 'index'])
loan_office_cert_assets['delegate_state'] = loan_office_cert_assets['delegate_state'].apply(lambda x: 'Constitution' if pd.isnull(x) else x)

In [33]:
loan_office_cert_assets

,delegate name,delegate_state,match name,sign,State,Year,Month,Day,Face Value,Specie Value,notes,original text,index
0,Roger Sherman,Constitution,Roger Sherman,yes,3,1777,10,21.0,300,257.02920,NaN,NaN,19227
1,Roger Sherman,Constitution,Roger Sherman,yes,3,1777,10,21.0,200,171.35280,NaN,NaN,19228
2,Roger Sherman,Constitution,Roger Sherman,yes,3,1777,10,21.0,200,171.35280,NaN,NaN,19229
3,Roger Sherman,Constitution,Roger Sherman,yes,3,1777,12,24.0,300,210.88330,NaN,NaN,19418
4,Roger Sherman,Constitution,Roger Sherman,yes,3,1778,9,26.0,300,66.08333,NaN,NaN,21796
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,William Sommersall,SC,William Somersall,yes,2,1780,6,29.0,1000,25.00000,NaN,William Dunscomb by William Somersall,16257
0,Thomas Dawes,MA,Thomas Dawes,yes,2,1777,11,21.0,1000,778.45830,NaN,Jn Gray & Thomas Dawes Trustees to Sach Green,5564
1,Thomas Dawes,MA,Thomas Dawes,yes,2,1777,11,21.0,1000,778.45830,NaN,Jn Gray & Thomas Dawes Trustees to Sach Green,5565
2,Thomas Dawes,MA,Thomas Dawes,yes,2,1777,11,21.0,1000,778.45830,NaN,Jn Gray & Thomas Dawes Trustees to Sach Green,5566


In [34]:
loan_office_cert_assets = pd.merge(cum_firstlast, loan_office_cert_assets, left_on = 'Full Name', right_on = 'delegate name').drop('Full Name', axis = 1).drop_duplicates()
loan_office_cert_assets_agg = loan_office_cert_assets.groupby(['delegate name', 'First Name', 'Last Name', 'delegate_state', 'sign'])[['Face Value', 'Specie Value']].agg(['sum','count'])
loan_office_cert_assets_agg

Face Value        \
                                                                 sum count   
delegate name    First Name Last Name delegate_state sign                    
Aaron Austin     Aaron      Austin    CT             yes        3700     3   
Aaron Wood       Aaron      Wood      MA             no          600     1   
Abraham Faw      Abraham    Faw       MD             yes         400     1   
Abraham Holmes   Abraham    Holmes    MA             no        13200     8   
Abraham Smith    Abraham    Smith     MA             no          300     1   
...                                                              ...   ...   
William White    William    White     VA             no        11400    18   
William Williams William    Williams  CT             yes       15600    28   
William Wilson   William    Wilson    PA             yes        2800     9   
William Woodhull William    Woodhull  NJ             yes        1000     2   
Williams John    Williams   John      RI             no       107800   176   

                                                          Specie Value        
                                                                   sum count  
delegate name    First Name Last Name delegate_state sign                     
Aaron Austin     Aaron      Austin    CT             yes     180.06667     3  
Aaron Wood       Aaron      Wood      MA             no       66.72500     1  
Abraham Faw      Abraham    Faw       MD             yes      35.53889     1  
Abraham Holmes   Abraham    Holmes    MA             no      527.52500     8  
Abraham Smith    Abraham    Smith     MA             no      123.49170     1  
...                                                                ...   ...  
William White    William    White     VA             no     1197.58450    18  
William Williams William    Williams  CT             yes    3957.08889    28  
William Wilson   William    Wilson    PA             yes     272.00277     9  
William Woodhull William    Woodhull  NJ             yes      51.06944     2  
Williams John    Williams   John      RI             no     9740.07502   176  

[494 rows x 4 columns]

# Pierce Certificates

In [35]:
#prepare pierce certificates
pierce = pd.read_excel("Data/Pre1790/Pierce_Certs_cleaned_2019.xlsx")

#clean first and last names
pierce = pierce[pierce['First'].apply(lambda x: not pd.isnull(x))]
pierce_ind = pierce[pierce['Last'].apply(lambda x: type(x) != str)].index
#fix issue with types for names
pierce.loc[pierce_ind, 'Last'] = ['True'] * len(pierce_ind)
#create full name
pierce['full name 1'] = pierce['First'] + " " + pierce['Last']

In [36]:
pierce_small_f1 = pierce[['State','full name 1', 'Value', 'CN']]
pierce_cert_assets = pd.merge(total_assets_final,
                              pierce_small_f1[pierce_small_f1['full name 1'].apply(lambda x: not pd.isnull(x))],
                              left_on = 'match name', right_on = 'full name 1').drop('full name 1', axis = 1)
pierce_cert_assets = pierce_cert_assets.drop_duplicates(['delegate name', 'CN'])
pierce_cert_assets['delegate_state'] = pierce_cert_assets['delegate_state'].apply(lambda x: 'Constitution' if pd.isnull(x) else x)

In [37]:
pierce_cert_assets = pd.merge(cum_firstlast, pierce_cert_assets, left_on = 'Full Name', right_on = 'delegate name').drop('Full Name', axis = 1).drop_duplicates()
pierce_cert_assets_agg = pierce_cert_assets.groupby(['delegate name', 'First Name', 'Last Name', 'delegate_state', 'sign'])[['Value']].agg(['sum','count'])
pierce_cert_assets_agg

Value      
                                                                 sum count
delegate name      First Name Last Name delegate_state sign               
A. Fisher          A.         Fisher    MA             yes    199.10     2
Aaron Hall         Aaron      Hall      NH             yes     67.20     2
Aaron Wood         Aaron      Wood      MA             no     120.60     2
Abraham Baldwin    Abraham    Baldwin   Constitution   yes   3746.75     6
Abraham Smith      Abraham    Smith     MA             no     290.20     5
...                                                              ...   ...
William White      William    White     VA             no    4272.54    26
William Williams   William    Williams  CT             yes   4062.61    49
William Wilson     William    Wilson    PA             yes   4800.33    36
Williams John      Williams   John      RI             no    6900.62    64
Zachariah Johnston Zachariah  Johnston  VA             yes    147.20     1

[550 rows x 2 columns]

# Marine Certificates

In [38]:
#Read in file
marine = pd.read_excel("Data/Pre1790/Marine_Liquidated_Debt_Certificates.xlsx", header = 11)
cols = marine.columns = ["Page","JPEG number","Number","Letter","Date of the Certificate: Month",
                         "Date of the Certificate: Day","Date of the Certificate: Year","First name","Last name",
                         "Title","Time when the debt became due: Month","Time when the debt became due: Day",
                         "Time when the debt became due: Year","Dollars","90th","Total Dollars_1","Total Dollars_2",
                         "Line Strike Thorugh: Yes?","Line Strike Thorugh: Note","Notes"]
marine.columns = cols

marine = marine[marine['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
marine['full name 1'] = marine['First name'] + " " + marine['Last name']

In [39]:
marine_small_f1 = marine[['full name 1', 'Number', 'Dollars', '90th']]
marine_cert_assets = pd.merge(total_assets_final,
                              marine_small_f1[marine_small_f1['full name 1'].apply(lambda x: not pd.isnull(x))],
                              left_on = 'match name', right_on = 'full name 1').drop('full name 1', axis = 1)
marine_cert_assets = marine_cert_assets.drop_duplicates(['delegate name', 'Number'])
marine_cert_assets['delegate_state'] = marine_cert_assets['delegate_state'].apply(lambda x: 'Constitution' if pd.isnull(x) else x)


In [40]:
marine_cert_assets = pd.merge(cum_firstlast, marine_cert_assets, left_on = 'Full Name', right_on = 'delegate name').drop('Full Name', axis = 1).drop_duplicates()
marine_cert_assets_agg = marine_cert_assets.groupby(['delegate name', 'First Name', 'Last Name', 'delegate_state', 'sign'])[['Dollars', '90th']].agg(['sum','count'])
marine_cert_assets_agg

Dollars  \
                                                                        sum   
delegate name          First Name     Last Name delegate_state sign           
Benjamin Browne        Benjamin       Browne    MA             yes    141.0   
Brown John             Brown          John      RI             yes    262.0   
Clark John I           Clark          John I    RI             yes     71.0   
David Bell             David          Bell      VA             no     108.0   
David Rice             David          Rice      NC             yes     76.0   
Davis John             Davis          John      RI             yes     35.0   
George Houser          George         Houser    NC             yes   1047.0   
James Neal             James          Neal      MA             no      42.0   
John Brown             John           Brown     NC             no     262.0   
John Davis             John           Davis     MA             yes     35.0   
John Dunlap            John           Dunlap    MA             yes      1.0   
John Elliot            John           Elliot    GA             yes    194.0   
John Hunter            John           Hunter    SC             yes    310.0   
John Jones             John           Jones     DE             yes     77.0   
John K. Smith          John K.        Smith     MA             yes     23.0   
John Rhea              John           Rhea      NC             yes     43.0   
John Smith             John           Smith     NY             yes     23.0   
John Taylor            John           Taylor    MA             no      72.0   
John Taylor Gilman     John Taylor    Gilman    NH             yes     72.0   
John Williams          John           Williams  VA             yes     28.0   
Jonathan Smith         Jonathan       Smith     MA             no     106.0   
Nathaniel Rogers       Nathaniel      Rogers    NH             yes     69.0   
Samuel Johnston        Samuel         Johnston  NC             yes     52.0   
Thomas Jones           Thomas         Jones     MA             yes     95.0   
William Hart           William        Hart      CT             yes     45.0   
William Jones          William        Jones     MA             no    1871.0   
William Morris         William        Morris    MD             yes    471.0   
William Samuel Johnson William Samuel Johnson   Constitution   yes     52.0   
William Scott          William        Scott     SC             yes     84.0   
William Smith          William        Smith     SC             yes    636.0   
Williams John          Williams       John      RI             no      28.0   

                                                                           \
                                                                    count   
delegate name          First Name     Last Name delegate_state sign         
Benjamin Browne        Benjamin       Browne    MA             yes      1   
Brown John             Brown          John      RI             yes      1   
Clark John I           Clark          John I    RI             yes      1   
David Bell             David          Bell      VA             no       1   
David Rice             David          Rice      NC             yes      1   
Davis John             Davis          John      RI             yes      1   
George Houser          George         Houser    NC             yes      1   
James Neal             James          Neal      MA             no       1   
John Brown             John           Brown     NC             no       1   
John Davis             John           Davis     MA             yes      1   
John Dunlap            John           Dunlap    MA             yes      1   
John Elliot            John           Elliot    GA             yes      1   
John Hunter            John           Hunter    SC             yes      1   
John Jones             John           Jones     DE             yes      2   
John K. Smith          John K.        Smith     MA             yes      

# Connecticut

In [41]:
#Read in file
CT = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_CT.xlsx", header = 11)
cols = CT.columns = ["Register Page","JPEG number","Number","Letter","Date of the Certificate: Month",
                     "Date of the Certificate: Day","Date of the Certificate: Year","First name","Last name",
                     "Title","First name 2","Last name 2","Title 2","Time when the debt became due: Month",
                     "Time when the debt became due: Day", "Time when the debt became due: Year","Dollars","90th",
                     "Line Strike Thorugh: Yes?","Line Strike Thorugh: Note","Notes"]
CT.columns = cols
CT = CT[CT['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
CT['full name 1'] = CT['First name'] + " " + CT['Last name']
CT['full name 2'] = CT['First name 2'] + " " + CT['Last name 2']

In [42]:
CT_short = CT[['full name 1', 'full name 2', 'Number', 'Dollars', '90th']]
CT_short['Number'] = CT_short['Number'].apply(lambda x: 'CT_' + str(int(x)))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Delaware

In [43]:
#Read in file
DE = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_DE.xlsx", header = 11)
cols = DE.columns = ["JPEG number","Number","Letter","Date of the Certificate: Month",
                     "Date of the Certificate: Day","Date of the Certificate: Year","Title","First name","Last name",
                     "Time when the debt became due: Month",
                     "Time when the debt became due: Day", "Time when the debt became due: Year","Dollars","90th",
                     "Line Strike Through","Note"]
DE.columns = cols

DE = DE[DE['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
DE['full name 1'] = DE['First name'] + " " + DE['Last name']

In [44]:
DE_short = DE[['full name 1', 'Number', 'Dollars', '90th']]
DE_short['Number'] = DE_short['Number'].apply(lambda x: 'DE_' + str(int(x)))
states_short = pd.concat([DE_short, CT_short])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


# Massachusetts

In [45]:
#Read in file
MA = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_MA.xlsx", header = 11)
cols = ["JPEG number","Page", "Number","Letter","Date of the Certificate: Month",
                     "Date of the Certificate: Day","Date of the Certificate: Year","First name","Last name","Title",
                     "First name 2","Last name 2", "Title 2","Time when the debt became due: Month",
                     "Time when the debt became due: Day", "Time when the debt became due: Year","Dollars","90th",
                     "Line Strike Through","Note"]
MA = MA[MA.columns[0:20]]
MA.columns = cols

MA = MA[MA['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
MA['full name 1'] = MA['First name'] + " " + MA['Last name']
MA['full name 2'] = MA['First name 2'] + " " + MA['Last name 2']

In [46]:
MA_short = MA[['full name 1', 'full name 2', 'Number', 'Dollars', '90th']]
MA_short['Number'] = MA_short['Number'].apply(lambda x: 'MA_' + str(int(x)))
states_short = pd.concat([states_short, MA_short])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


# New Hampshire

In [47]:
#Read in file
NH = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_NH.xlsx", header = 11)
cols = ["Page","JPEG number","Number","Letter","Date of the Certificate: Month",
                         "Date of the Certificate: Day","Date of the Certificate: Year","First name","Last name",
                         "Title","Time when the debt became due: Month","Time when the debt became due: Day",
                         "Time when the debt became due: Year","Dollars","90th",
                         "Line Strike Thorugh: Yes?","Line Strike Thorugh: Note","Notes"]
NH.columns = cols

NH = NH[NH['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
NH['full name 1'] = NH['First name'] + " " + NH['Last name']

In [48]:
NH_short = NH[['full name 1', 'Number', 'Dollars', '90th']]
NH_short['Number'] = NH_short['Number'].apply(lambda x: 'NH_' + str(int(x)))
states_short = pd.concat([states_short, NH_short])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


# New Jersey

In [49]:
#Read in file
NJ = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_NJ.xlsx", header = 10)
cols = ["JPEG number","Number","Letter","Date of the Certificate: Month",
        "Date of the Certificate: Day","Date of the Certificate: Year","Title","First name","Last name",
        "Title 2","First name 2","Last name 2","Time when the debt became due: Month",
        "Time when the debt became due: Day", "Time when the debt became due: Year","Dollars","90th",
        "Strike Through Number","Note"]
NJ = NJ[NJ.columns[0:19]]
NJ.columns = cols

NJ = NJ[NJ['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
NJ['full name 1'] = NJ['First name'] + " " + NJ['Last name']
NJ['full name 2'] = NJ['First name 2'] + " " + NJ['Last name 2']

In [50]:
NJ_short = NJ[['full name 1', 'full name 2', 'Number', 'Dollars', '90th']]
NJ_short = NJ_short[NJ_short['Number'].apply(lambda x: not pd.isnull(x))]
NJ_short['Number'] = NJ_short['Number'].apply(lambda x: 'NJ_' + str(int(x)))
states_short = pd.concat([states_short, NJ_short])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


# New York

In [51]:
#Read in file
NY = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_NY.xlsx", header = 10)
cols = ["Page", "JPEG number","Number","Letter","Date of the Certificate: Month",
        "Date of the Certificate: Day","Date of the Certificate: Year","First name","Last name",
        "Title","First name 2","Last name 2","Title 2","Time when the debt became due: Month",
        "Time when the debt became due: Day", "Time when the debt became due: Year","Dollars","90th",
        "Strike Through Yes?","Note", "Notes"]
NY = NY[NY.columns[0:21]]
NY.columns = cols

NY = NY[NY['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
NY['full name 1'] = NY['First name'] + " " + NY['Last name']
NY['full name 2'] = NY['First name 2'] + " " + NY['Last name 2']

In [52]:
NY_short = NY[['full name 1', 'full name 2', 'Number', 'Dollars', '90th']]
NY_short = NY_short[NY_short['Number'].apply(lambda x: not pd.isnull(x))]
NY_short['Number'] = NY_short['Number'].apply(lambda x: 'NY_' + str(int(x)))
states_short = pd.concat([states_short, NY_short])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


# Pennsylvania P1

In [53]:
#Read in file
PA = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_PA_story.xlsx", header = 11)
cols = ["JPEG number","Number","Letter","Date of the Certificate: Year",
        "Date of the Certificate: Month","Date of the Certificate: Date","Title","First name","Last name","Time when the debt became due: Year",
        "Time when the debt became due: Month","Time when the debt became due: Day",
        "Dollars","90th","10th", "Exchange","Amount in Specie_1", "Amount in Specie_2","Line Strike Thorugh?","Notes"]
PA = PA[PA.columns[1:21]]
PA.columns = cols

PA = PA[PA['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
PA['full name 1'] = PA['First name'] + " " + PA['Last name']

In [54]:
PA_short_1 = PA[['full name 1', 'Number', 'Dollars', '90th']]
PA_short_1 = PA_short_1[PA_short_1['Number'].apply(lambda x: not pd.isnull(x))]
PA_short_1['Number'] = PA_short_1['Number'].apply(lambda x: 'PA_1_' + str(int(x)))
states_short = pd.concat([states_short, PA_short_1])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


# Pennsylvania P2

In [55]:
#Read in file
PA2 = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_PA_stelle.xlsx", header = 11)
cols = ["JPEG number","Number","Letter","Date of the Certificate: Year",
        "Date of the Certificate: Month","Date of the Certificate: Date","Title","First name","Last name","Title 2","First name 2","Last name 2",
        "Time when the debt became due: Year", "Time when the debt became due: Month","Time when the debt became due: Day",
        "Dollars","90th","Line Strike Thorugh?","Notes"]
PA2 = PA2[PA2.columns[1:21]]
PA2.columns = cols

PA2 = PA2[PA2['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
PA2['full name 1'] = PA2['First name'] + " " + PA2['Last name']
PA2['full name 2'] = PA2['First name 2'] + " " + PA2['Last name 2']

In [56]:
PA_short_2 = PA2[['full name 1', 'full name 2', 'Number', 'Dollars', '90th']]
PA_short_2 = PA_short_2[PA_short_2['Number'].apply(lambda x: not pd.isnull(x))]
PA_short_2['Number'] = PA_short_2['Number'].apply(lambda x: 'PA_2_' + str(int(x)))
states_short = pd.concat([states_short, PA_short_2])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


# Rhode Island

In [57]:
#Read in file
RI = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_RI.xlsx", header = 12)
cols =  ["Page", "JPEG number","Number","Letter","Date of the Certificate: Month",
        "Date of the Certificate: Day","Date of the Certificate: Year","First name","Last name",
        "Title","First name 2","Last name 2","Title 2","Time when the debt became due: Month",
        "Time when the debt became due: Day", "Time when the debt became due: Year","Dollars","90th",
        "8th"]
RI = RI[RI.columns[0:19]]
RI.columns = cols

RI = RI[RI['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
RI['full name 1'] = RI['First name'] + " " + RI['Last name']
RI['full name 2'] = RI['First name 2'] + " " + RI['Last name 2']

In [58]:
RI_short = RI[['full name 1', 'full name 2', 'Number', 'Dollars', '90th']]
RI_short = RI_short[RI_short['Number'].apply(lambda x: not pd.isnull(x))]
RI_short['Number'] = RI_short['Number'].apply(lambda x: 'RI_' + str(int(x)))
states_short = pd.concat([states_short, RI_short])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


# Final Steps

In [59]:
state_cert_assets = pd.merge(total_assets_final,
                                   states_short[states_short['full name 1'].apply(lambda x: not pd.isnull(x))],
                                   left_on = 'match name', right_on = 'full name 1').drop('full name 1', axis = 1)
add_to_assets = pd.merge(total_assets_final,
                         states_short[states_short['full name 2'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 2').drop('full name 2', axis = 1)
state_cert_assets = pd.concat([state_cert_assets, add_to_assets])
state_cert_assets = state_cert_assets.drop_duplicates(['delegate name', 'Number'])
state_cert_assets['delegate_state'] = state_cert_assets['delegate_state'].apply(lambda x: 'Constitution' if pd.isnull(x) else x)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [60]:
state_cert_assets = pd.merge(cum_firstlast, state_cert_assets, left_on = 'Full Name', right_on = 'delegate name').drop('Full Name', axis = 1).drop_duplicates()
state_cert_assets_agg = state_cert_assets.groupby(['delegate name', 'First Name', 'Last Name',  'delegate_state', 'sign'])[['Dollars', '90th']].agg(['sum','count'])
state_cert_assets_agg

Dollars        \
                                                                sum count   
delegate name      First Name Last Name delegate_state sign                 
Abraham Smith      Abraham    Smith     MA             no       4.0     1   
Abraham Stout      Abraham    Stout     PA             yes     14.0     2   
Albert Baker       Albert     Baker     NY             no      28.0     1   
Alexander Hamilton Alexander  Hamilton  Constitution   yes    160.0     1   
Alexander Martin   Alexander  Martin    Constitution   no     124.0     2   
...                                                             ...   ...   
William White      William    White     VA             no     721.0    12   
William Williams   William    Williams  CT             yes   3552.0    14   
William Wilson     William    Wilson    PA             yes     96.0     3   
Williams John      Williams   John      RI             no    1766.0    19   
Zephaniah Platt    Zephaniah  Platt     NY             yes    284.0     1   

                                                            90th        
                                                             sum count  
delegate name      First Name Last Name delegate_state sign             
Abraham Smith      Abraham    Smith     MA             no      0     0  
Abraham Stout      Abraham    Stout     PA             yes   136     2  
Albert Baker       Albert     Baker     NY             no      0     0  
Alexander Hamilton Alexander  Hamilton  Constitution   yes     0     0  
Alexander Martin   Alexander  Martin    Constitution   no    110     2  
...                                                          ...   ...  
William White      William    White     VA             no    383    10  
William Williams   William    Williams  CT             yes   486     9  
William Wilson     William    Wilson    PA             yes     6     1  
Williams John      Williams   John      RI             no    579    16  
Zephaniah Platt    Zephaniah  Platt     NY             yes    26     1  

[460 rows x 4 columns]

In [62]:
loan_office_cert_assets_agg.to_csv('Producables/Loan_Office_Certificates_Matched.csv')
pierce_cert_assets_agg.to_csv('Producables/Pierce_Certificates_Matched.csv')
marine_cert_assets_agg.to_csv('Producables/Marine_Certificates_Matched.csv')
state_cert_assets_agg.to_csv('Producables/State_Certificates_Matched.csv')